In [18]:
import re
from collections import defaultdict
import math
import operator

** Xây dụng class N-gram tính xác suất và lưu trữ N-gram**

In [19]:
# Xây dựng class N-gram
class N_Gram:
    def __init__(self, n=2, trie = None ): # đưa cây trie đã được train và n-gram cụ thể 
        self.n = n
        self.alpha = 1.0    # dùng cho khi nào sác xuất đầu ra là 0
        self.prob_ngram = {}        
        self.count_n = {}
        self.trie = trie
        
    
    def train(self, sentense):  # Hàm này sẽ train corpus có type là  list(String)  cụ thể là một list chứa các token liên tiếp nhau

      for i in range(len(sentense)):

          # Lấy các lần xuất hiện của bigram   
          if i+1<len(sentense):
          
              if ((sentense[i],sentense[i+1]) in self.count_n ):
                  self.count_n[(sentense[i],sentense[i+1])] += 1 # nó được lưu trữ theo tuần tự
              else:
                  self.count_n[(sentense[i],sentense[i+1])] = 1
  # Tính xác suất cho từng biagram
        
      for token in self.count_n:
          self.prob_ngram[token] =  self.Prob_Ngram(token)
            
            
    def Prob_Ngram(self, token): # đầu vào là các token của  N-gram và tính xác suất xuất hiện của nó
    # sữ dụng smoothing
    # Xác suất tính bằng só lần xuất hiện của N-gram đó chia cho số lần xuất hiện thông thường của unigram
        return (self.count_n[token] + self.alpha )/(self.trie.FindCount(self.trie.root,token[0]) + trie.V)
          
        # Tính độ phức tạp
    def Calculate_Perlexity (self, text):
        words = re.findall(r'\b\w+\b', text.lower())
        perlexity = 1
        biagram_prob = 1
        for i in range(1,len(text)):
            if ((words[i-1],words[i]) in self.prob_ngram):
                biagram_prob *= self.prob_ngram[(words[i-1],words[i])]
            else:
                if words[i-1] in self.count_n:
                    biagram_prob =biagram_prob * (1/(self.count_n[words[i-1]] + self.V  ))
                else:
                    biagram_prob *=0.000000001
                    
        perlexity  *= 1/biagram_prob
        perplexity = math.pow(perplexity, 1.0 / len(words))
        
        return perplexity
        

Xây dựng cấu trúc dữ liệu cây Trie đẻ lưu trữ và tìm kiếm *unigram* 

In [20]:
class TrieNode:
  def __init__(self):
      self.children = {}
      self.end_word = False
      self.value = None
      self.count = 0
      

class Trie:
  def __init__(self):
      self.root = TrieNode()
      self.V = 0
  # Hàm thêm token vào
  def Insert(self, x, key):      
    for i in range(0, len(key)):
        if key[i] not in x.children:
            x.children[key[i]] = TrieNode()
            x = x.children[key[i]]
        else:

            x = x.children[key[i]]
    if x.end_word == False:
      self.V +=1
    
    x.end_word = True
    x.value = key
    x.count +=1
  # Hàm tìm kiếm giá trị đếm của token
  def FindCount(self,x, key):
    for i in range(0, len(key)):
        if key[i] not in x.children:
            return False
        else:
            x = x.children[key[i]]
    if x.end_word:
        return x.count
    else:
        return False
  
  # Hàm tìm kiếm token có tồn tại hay không
  def FindWord(self,x, key):
    
    for i in range(0, len(key)):
        if key[i] not in x.children:
            return False
        else:
            x = x.children[key[i]]
    if x.end_word:
        return x.value
    else:
        return False

  # Hàm xóa token nếu token đó không phù hợp
  def Delete(self,x, key):
    if len(key) == 0:
        if x.end_word == True:
            x.end_word = False
            x.value = None
        for i in range(len(x.children)):
            if key[i] in x.children:
                return x # Nếu trong từ này còn có các chuỗi khác thì trả về x
        return None # còn không thì xóa uôn nhánh key
    x.children[key[0]] = self.Delete(x.children[key[0]], key[1:])
    return x

# Phương thức in ra màn hình cho biết các token được viết vào thành công
  def __str__(self):
    return str(self._get_words(self.root))

  def _get_words(self, x):
    words = []
    if x.end_word:
        words.append(x.value)
    for c in x.children:
        words.extend(self._get_words(x.children[c]))
    return words


# Phương thức tìm kiếm word error và đưa ra các từ đại diện, khi dùng hàm này thì từ đó chắc chăn phải lỗi
  def FindSpelling(self,x, key):
    candidates = set()
    for i in range(0,len(key)):
      if key[i] not in x.children: # xảy ra ba trường hợp là lỗ thiếu chử, lỗ thêm chữ và lỗi chữ bị thay đổi
        deletion =  key[:i]+ key[i+1:]
        candidates.add(deletion)

        for c in x.children:
          subtution = key[:i] + c + key[i+1:]
          candidates.add(subtution)

        for c in x.children:
          insertion = key[:i] + c + key[i:]
          candidates.add(insertion)
        break
      else:
        
        x =  x.children[key[i]]
   

    accurate_candidates = set()
    
    for candidate in candidates:
        if self.FindWord(self.root, candidate):
            accurate_candidates.add(candidate)
    return accurate_candidates # trả về set
  

    

Dùng Edit Distance để tính khoảng cách giữa hai từ với nhau

In [21]:

def edit_distance(s1, s2):
    dp = [[0 for j in range(len(s2) + 1)] for i in range(len(s1) + 1)]
    for i in range(1, len(s2) + 1):
        dp[0][i] = i

    for i in range(1, len(s1) + 1):
        dp[i][0] = i

    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            dp[i][j] = min(min(dp[i - 1][j], dp[i][j - 1]) + 1, dp[i - 1][j - 1] + (1 if s1[i - 1] != s2[j - 1] else 0))

    return dp[len(s1)][len(s2)]



# Hàm chỉnh sữa thêm words trả về set các từ đã được chỉnh sưa từ từ bị sai lỗi chính tả 

def edits1(word):
    """Tất cả các ký tự cơ bản"""
    """ Ý tưởng dùng xóa, thêm, chỉnh, đổi vị trí để tạo ra từ mới cho việc tìm kiếm chuyển đổi của word """
    
    letters = ["a","â","ă","b","c","d","đ","e","ê","g","h","i","k","l","m","n","o","ô","ơ","p","q","r","s","t",
               "u","ư","v","x","y"]
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]


    deletes = [L + R[1:] for L, R in splits if R]

    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]

    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]

    inserts = [L + c + R for L, R in splits for c in letters]

    return set(deletes + transposes + replaces + inserts)


Xây dựng class Spelling corection có các phương thức tìm lỗi chính tả và gợi ý từ phù họp

In [45]:


class SpellingCorrection(N_Gram):
    def __init__ (self, n_gram =2, trie = None):
      super().__init__(n_gram, trie)
      
    
    def ZeroProb(self, twotoken):
      return (self.alpha )/(self.trie.FindCount(self.trie.root,token[0]) + trie.V)

    def SuggestCandidates(self, sentences): # sentenses is type String, type ouput is list[set], list[int], list[string]
      token = sentences.split()
      candidates =  []
      indexError = []
      for i in range(len(token)):
        if self.trie.FindWord(self.trie.root, token[i]):
          pass
        else:
          candidates.append(self.trie.FindSpelling(self.trie.root, token[i]))
          indexError.append(i)

          print(" Chữ "+ token[i]  + " bị sai lỗi chính tả, nó nằm ở vị trí thứ "+str(i+1)+ "\n")
          print(token)
      return candidates, indexError, token


 

    def ProbSentences(self, token, index): # token is type  list[string], return prob
      if index == 0:
          if (token[index], token[index + 1]) in self.prob_ngram:
            return math.log10( 1/self.prob_ngram[ (token[index], token[index + 1])] )
          else:
            return math.log10(1/self.ZeroProb((token[index], token[index + 1])))

      else:
          if index + 1 == len(token):
              if (token[index-1], token[index]) in self.prob_ngram:
                return math.log10( 1/self.prob_ngram[ (token[index-1], token[index]) ]) 
              else:
                return math.log10(1/self.ZeroProb((token[index-1], token[index])))
          else:
              if (token[index-1], token[index]) in self.prob_ngram:
                a=self.prob_ngram[(token[index-1],token[index])]
              else:
                a= self.ZeroProb((token[index-1], token[index]))
              if (token[index], token[index + 1]) in self.prob_ngram:
                b= self.prob_ngram[(token[index],token[index + 1])]
              else:
                b= self.ZeroProb((token[index], token[index + 1]))
              print(a)
              print(b)
              return math.log10( 1/(a*b))
      raise KeyError("Probabilities not found for given tokens")

      
    
    def SuggestProbToken(self, sentences):
      candidates, indexError, token  =  self.SuggestCandidates(sentences)
      # Kiểm tra xem giá trị xác suất của trước và sau của từ này 
      prob_candidates = {}
      for index in range(len(candidates)):
        for realtoken in candidates[index]:
          token[indexError[index]] =  realtoken
          print(token)
          prob_candidates[realtoken]  = self.ProbSentences(token, indexError[index])
      return prob_candidates, indexError, token

    # DE XUAT TU DUNG
    def SuggestSentences(self, sentences):
      prob_candidates, indexError, token =  self.SuggestProbToken(sentences.lower())
      print(prob_candidates)
      for i in range(len(indexError)):
        try:
          min_value_key = min(prob_candidates.items(), key=operator.itemgetter(1))[0]
        except KeyError:
          print("Từ điển trống. Không tìm thấy giá trị nhỏ nhất.")
          break
        token[indexError[i]]  =  min_value_key
      
      realSentences =  " ".join(token).capitalize()
      return realSentences


Đọc dữ liệu và train

In [23]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
file_path = '/content/drive/MyDrive/RealWord/data_news.txt'
with open(file_path, 'r', encoding = "utf-8") as f:
    file_contents = f.read().lower()

In [25]:
corpus = file_contents.split()

In [46]:
trie = Trie()
for token in corpus:
  trie.Insert(trie.root,token)

# train cho N-gram

In [47]:
spellingCheck  = SpellingCorrection(2,trie)
spellingCheck.train(corpus)

In [28]:
spellingCheck.prob_ngram[("hà","thành")]

0.002949215940152496

In [48]:
print(spellingCheck.SuggestSentences("Tôi biet cô ấy"))

 Chữ biet bị sai lỗi chính tả, nó nằm ở vị trí thứ 2

['tôi', 'biet', 'cô', 'ấy']
['tôi', 'birt', 'cô', 'ấy']
7.875876191226274e-05
7.875876191226274e-05
['tôi', 'biệt', 'cô', 'ấy']
7.875876191226274e-05
7.875876191226274e-05
['tôi', 'biết', 'cô', 'ấy']
0.0010429859196900843
0.00047564041584562073
{'birt': 8.207402239179029, 'biệt': 8.207402239179029, 'biết': 6.304442804372275}
Tôi biết cô ấy


In [30]:
spellingCheck.prob_ngram[('yêu', 'cô')]

0.00014847809948032666

In [31]:
if ("tôi","yêu") in spellingCheck.prob_ngram:
  print(1)

1
